# Statistical Analysis & Model Evaluation

**Purpose:**
Perform rigorous statistical evaluation of machine learning models to validate findings for publication.

**Steps:**
1.  **Load Data:** Read `df_baseline_processed.csv` and `df_longitudinal_processed.csv`.
2.  **Baseline Statistical Analysis:** Calculate 95% Confidence Intervals (Bootstrap) and McNemar's Tests for HC vs MCI diagnosis.
3.  **Longitudinal Statistical Analysis:** Calculate 95% Confidence Intervals for prognosis prediction (Accuracy).
4.  **Reporting:** Print statistical results.

In [8]:
# Imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict, LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix
from statsmodels.stats.contingency_tables import mcnemar

# Set random seed
np.random.seed(42)

# Plot settings
sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', None)

# Paths
PROCESSED_PATH = "data/processed/"
OUT_FIGS = "report/figs/"
os.makedirs(OUT_FIGS, exist_ok=True)

BASELINE_FILE = "df_baseline_processed.csv"
LONGITUDINAL_FILE = "df_longitudinal_processed.csv"

def save_and_show(fig, out_fp):
    """Save matplotlib figure with tight layout and show it."""
    fig.tight_layout()
    fig.savefig(out_fp, bbox_inches='tight', dpi=300)
    plt.show()

In [9]:
# Statistical Helper Functions
def bootstrap_ci(y_true, y_pred, n_bootstraps=1000):
    """Calculate 95% Confidence Interval using Bootstrap sampling."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n = len(y_true)
    accuracy_scores = []
    
    for _ in range(n_bootstraps):
        # Resample with replacement
        indices = np.random.choice(n, n, replace=True)
        if len(np.unique(y_true[indices])) < 2:
            # Skip useless samples
            continue
        acc = accuracy_score(y_true[indices], y_pred[indices])
        accuracy_scores.append(acc)
    
    mean_acc = np.mean(accuracy_scores)
    ci_low = np.percentile(accuracy_scores, 2.5)
    ci_high = np.percentile(accuracy_scores, 97.5)
    
    return mean_acc, ci_low, ci_high

def mcnemar_test(y_true, y_pred1, y_pred2):
    """Perform McNemar's Test for pairwise classifier comparison."""
    y_true = np.array(y_true)
    y_pred1 = np.array(y_pred1)
    y_pred2 = np.array(y_pred2)
    
    correct1 = (y_pred1 == y_true)
    correct2 = (y_pred2 == y_true)
    
    # Contingency Table
    #          M2 Correct  M2 Wrong
    # M1 Correct    a         b
    # M1 Wrong      c         d
    a = np.sum(correct1 & correct2)
    b = np.sum(correct1 & ~correct2)
    c = np.sum(~correct1 & correct2)
    d = np.sum(~correct1 & ~correct2)
    
    table = [[a, b], [c, d]]
    
    # Exact McNemar test (binomial)
    result = mcnemar(table, exact=True)
    return result.pvalue

In [10]:
# Load Data
print("Loading Processed Data...")
df_baseline = pd.read_csv(os.path.join(PROCESSED_PATH, BASELINE_FILE))
df_longitudinal = pd.read_csv(os.path.join(PROCESSED_PATH, LONGITUDINAL_FILE))

print(f"Baseline Shape: {df_baseline.shape}")
print(f"Longitudinal Shape: {df_longitudinal.shape}")

Loading Processed Data...
Baseline Shape: (82, 572)
Longitudinal Shape: (47, 1799)


# 1. Baseline Diagnosis - Statistical Analysis

Comparing algorithms (LR, RF, DT, SVM, NB, XGBoost) using 5-Fold Stratified CV.
**New:** Calculating 95% Confidence Intervals and McNemar's p-values.

In [11]:
# Prepare Data (Same as model.ipynb)
print(f'Available columns in Baseline: {df_baseline.columns.tolist()}')
target_col = 'Diagnosis_base'
if target_col not in df_baseline.columns:
    target_col = 'Diagnosis'

# Drop clear leakage columns
X_base = df_baseline.drop(columns=[target_col, 'Condition', 'Condition_clean'], errors='ignore')
y_base = df_baseline[target_col]

# Variance Threshold
feature_names = X_base.columns
selector = VarianceThreshold(threshold=0) 
X_base = selector.fit_transform(X_base)
feature_names = feature_names[selector.get_support()]

# Model Definitions
models_baseline = {
    'Logistic Regression': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))
    ]),
    'Decision Tree': Pipeline([
        ('clf', DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=42))
    ]),
    'Random Forest': Pipeline([
        ('clf', RandomForestClassifier(n_estimators=150, max_depth=6, random_state=42))
    ]),
    'SVM': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', SVC(kernel='linear', class_weight='balanced', random_state=42))
    ]),
    'Naive Bayes': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', GaussianNB())
    ]),
    'KNN': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', KNeighborsClassifier(n_neighbors=5))
    ]),
    'XGBoost': XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric='logloss',
        random_state=42
    )
}

# Statistical Evaluation Loop
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\nRunning Baseline Statistical Benchmark...\n")
results_base = {}
baseline_preds = {}

print(f"{'Model':<20} | {'Mean Acc':<10} | {'95% CI':<20}")
print("-" * 55)

for name, model in models_baseline.items():
    y_pred = cross_val_predict(model, X_base, y_base, cv=cv)
    baseline_preds[name] = y_pred
    
    # Bootstrap CI
    mean_acc, ci_low, ci_high = bootstrap_ci(y_base, y_pred)
    results_base[name] = mean_acc
    
    print(f"{name:<20} | {mean_acc:.4f}     | [{ci_low:.4f}, {ci_high:.4f}]")

# McNemar's Test (vs XGBoost)
print("\nPAIRWISE McNEMAR TEST (vs XGBoost)")
print("-" * 50)
xgb_pred = baseline_preds['XGBoost']

for name in baseline_preds:
    if name == 'XGBoost': continue
    p_val = mcnemar_test(y_base, xgb_pred, baseline_preds[name])
    sig = "Yes" if p_val < 0.05 else "No"
    print(f"XGBoost vs {name:<20} | p={p_val:.4f} | Sig: {sig}")

Available columns in Baseline: ['CVT_LPP_C3', 'CVT_LPP_C4', 'CVT_LPP_Cz', 'CVT_LPP_F3', 'CVT_LPP_F4', 'CVT_LPP_F7', 'CVT_LPP_F8', 'CVT_LPP_Fp1', 'CVT_LPP_Fp2', 'CVT_LPP_Fz', 'CVT_LPP_O1', 'CVT_LPP_O2', 'CVT_LPP_P3', 'CVT_LPP_P4', 'CVT_LPP_POz', 'CVT_LPP_Pz', 'CVT_LPP_T3', 'CVT_LPP_T4', 'CVT_LPP_T5', 'CVT_LPP_T6', 'CVT_LPPmaxLoc_C3', 'CVT_LPPmaxLoc_C4', 'CVT_LPPmaxLoc_Cz', 'CVT_LPPmaxLoc_F3', 'CVT_LPPmaxLoc_F4', 'CVT_LPPmaxLoc_F7', 'CVT_LPPmaxLoc_F8', 'CVT_LPPmaxLoc_Fp1', 'CVT_LPPmaxLoc_Fp2', 'CVT_LPPmaxLoc_Fz', 'CVT_LPPmaxLoc_O1', 'CVT_LPPmaxLoc_O2', 'CVT_LPPmaxLoc_P3', 'CVT_LPPmaxLoc_P4', 'CVT_LPPmaxLoc_POz', 'CVT_LPPmaxLoc_Pz', 'CVT_LPPmaxLoc_T3', 'CVT_LPPmaxLoc_T4', 'CVT_LPPmaxLoc_T5', 'CVT_LPPmaxLoc_T6', 'CVT_N1minLoc_C3', 'CVT_N1minLoc_C4', 'CVT_N1minLoc_Cz', 'CVT_N1minLoc_F3', 'CVT_N1minLoc_F4', 'CVT_N1minLoc_F7', 'CVT_N1minLoc_F8', 'CVT_N1minLoc_Fp1', 'CVT_N1minLoc_Fp2', 'CVT_N1minLoc_Fz', 'CVT_N1minLoc_O1', 'CVT_N1minLoc_O2', 'CVT_N1minLoc_P3', 'CVT_N1minLoc_P4', 'CVT_N1minLoc_

# 2. Final Baseline Model (Analysis)

XGBoost Confusion Matrix & Detailed Metrics.

In [12]:
y_pred_base = baseline_preds['XGBoost']
acc_base = accuracy_score(y_base, y_pred_base)
print(f"Baseline XGBoost Accuracy (CV): {acc_base:.4f}")

Baseline XGBoost Accuracy (CV): 0.8780


# 3. Longitudinal Prognosis - Statistical Analysis

Applying Bootstrap CI to LOOCV results for small dataset validation.

In [13]:
# Prepare Longitudinal Data (Same as model.ipynb)
target_long = 'MCI_progression_V1' if 'MCI_progression_V1' in df_longitudinal.columns else 'MCI_progression'
df_long_model = df_longitudinal.dropna(subset=[target_long]).copy()
# Binary target
df_long_model['target_binary'] = df_long_model[target_long].apply(lambda x: 0 if x < 0 else 1)
y_long = df_long_model['target_binary']

# Drop leakage
leakage_keywords = ['_V2', '_Diff', 'progression', 'target', 'Diagnosis_V2', 'MCI_score_V2', 'diff']
features_to_drop = [c for c in df_long_model.columns if any(k.lower() in c.lower() for k in leakage_keywords)]
X_long = df_long_model.drop(columns=features_to_drop, errors='ignore')

# Variance Threshold
selector = VarianceThreshold(threshold=0) 
X_long = selector.fit_transform(X_long)

# Model List
models_longitudinal = {
    'Logistic Regression': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', LogisticRegression(max_iter=200, class_weight='balanced'))
    ]),
    'Decision Tree': Pipeline([
        ('clf', DecisionTreeClassifier(max_depth=5, class_weight='balanced', random_state=42))
    ]),
    'Random Forest': Pipeline([
        ('clf', RandomForestClassifier(n_estimators=150, max_depth=6, random_state=42))
    ]),
    'SVM': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', SVC(kernel='linear', class_weight='balanced', random_state=42))
    ]),
    'Naive Bayes': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', GaussianNB())
    ]),
    'KNN': Pipeline([
        ('selector', SelectKBest(score_func=f_classif, k=10)),
        ('clf', KNeighborsClassifier(n_neighbors=5))
    ]),
    'XGBoost': XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric='logloss',
        random_state=42
    )
}

# Evaluation Loop (LOOCV + Bootstrap)
cv_loo = LeaveOneOut()
results_long = {}

print(f"{'Model':<20} | {'Mean Acc':<10} | {'95% CI':<20}")
print("-" * 55)

for name, model in models_longitudinal.items():
    # Use cross_val_predict for LOOCV to get concise predictions vector
    y_pred = cross_val_predict(model, X_long, y_long, cv=cv_loo)
    
    mean_acc, ci_low, ci_high = bootstrap_ci(y_long, y_pred)
    results_long[name] = mean_acc
    
    print(f"{name:<20} | {mean_acc:.4f}     | [{ci_low:.4f}, {ci_high:.4f}]")

Model                | Mean Acc   | 95% CI              
-------------------------------------------------------
Logistic Regression  | 0.5329     | [0.4043, 0.6809]
Decision Tree        | 0.4873     | [0.3404, 0.6383]
Random Forest        | 0.3591     | [0.2340, 0.4894]
SVM                  | 0.5955     | [0.4681, 0.7234]
Naive Bayes          | 0.3621     | [0.2340, 0.5106]
KNN                  | 0.6151     | [0.4681, 0.7452]
XGBoost              | 0.5107     | [0.3830, 0.6596]


# 4. Final Longitudinal Model (KNN)

Detailed breakdown of the best performing longitudinal model.

In [14]:
# Re-train best model for plotting context
model_long = Pipeline([
    ('selector', SelectKBest(score_func=f_classif, k=10)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

y_pred_long = cross_val_predict(model_long, X_long, y_long, cv=cv_loo)
mean_acc, ci_low, ci_high = bootstrap_ci(y_long, y_pred_long)

print(f"Final KNN Accuracy: {mean_acc:.4f} (95% CI: {ci_low:.4f} - {ci_high:.4f})")

Final KNN Accuracy: 0.6197 (95% CI: 0.4894 - 0.7660)
